In [19]:
import open3d as o3d
import open3d.core as o3c
import numpy as np
import matplotlib.pyplot as plt
import copy
import os
import sys

# Point cloud
This tutorial demonstrates basic usage of a point cloud.

## PointCloud creation
The first part of the tutorial shows how to construct a point cloud.

In [25]:
# Create a empty point cloud on CPU.
pcd = o3d.t.geometry.PointCloud()
print(pcd)
# Create a empty point cloud on GPU.
pcd = o3d.t.geometry.PointCloud(o3c.Device("CUDA:0"))
print(pcd)

# Create a point cloud from open3d tensor.
pcd = o3d.t.geometry.PointCloud(o3c.Tensor([[0, 0, 0], [1, 1, 1]], o3c.float32))
print(pcd)
# Create a point cloud from open3d tensor with float64.
pcd = o3d.t.geometry.PointCloud(o3c.Tensor([[0, 0, 0], [1, 1, 1]], o3c.float64))
print(pcd)
# Create a point cloud from open3d tensor on GPU.
pcd = o3d.t.geometry.PointCloud(o3c.Tensor([[0, 0, 0], [1, 1, 1]], o3c.float32, o3c.Device("CUDA:0")))
print(pcd)

# Special creation.
# Create a point cloud from numpy array.
pcd = o3d.t.geometry.PointCloud(np.array([[0, 0, 0], [1, 1, 1]], dtype=np.float32))
print(pcd)
# Create a point cloud from python list.
pcd = o3d.t.geometry.PointCloud([[0., 0., 0.], [1., 1., 1.]])
print(pcd)

# Error creation. The point cloud must have shape of (N, 3).
pcd = o3d.t.geometry.PointCloud(o3c.Tensor([0, 0, 0, 0], o3c.float32))

PointCloud on CPU:0 [0 points] Attributes: None.
PointCloud on CUDA:0 [0 points] Attributes: None.
PointCloud on CPU:0 [2 points (Float32)] Attributes: None.
PointCloud on CPU:0 [2 points (Float64)] Attributes: None.
PointCloud on CUDA:0 [2 points (Float32)] Attributes: None.
PointCloud on CPU:0 [2 points (Float32)] Attributes: None.
PointCloud on CPU:0 [2 points (Float64)] Attributes: None.


RuntimeError: [1;31m[Open3D Error] (open3d::t::geometry::PointCloud::PointCloud(const open3d::core::Tensor&)) /root/Open3D/cpp/open3d/t/geometry/PointCloud.cpp:72: Tensor has shape {4}, but is expected to have compatible with {None, 3}.
[0;m

The `PointCloud` can be created on both CPU and GPU, and with different data types. The device of the point cloud will be the same as the device of the input tensor.

Besides, `PointCloud` can be also created by `TensorMap` (concel) or python dict with multiple attributes. 

In [42]:
map_to_tensors = o3d.t.geometry.TensorMap('positions')
# The `positions` attribute must be specified.
map_to_tensors['positions'] = o3c.Tensor([[0, 0, 0], [1, 1, 1]], o3c.float32)
map_to_tensors['normals'] = o3c.Tensor([[0, 0, 1], [0, 0, 1]], o3c.float32)
map_to_tensors['labels'] = o3c.Tensor([0, 1], o3c.int64)

pcd = o3d.t.geometry.PointCloud(map_to_tensors)
print(pcd)



TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. open3d.cuda.pybind.t.geometry.PointCloud(device: open3d.cuda.pybind.core.Device = CPU:0)
    2. open3d.cuda.pybind.t.geometry.PointCloud(positions: open3d.cuda.pybind.core.Tensor)
    3. open3d.cuda.pybind.t.geometry.PointCloud(map_keys_to_tensors: Dict[str, open3d.cuda.pybind.core.Tensor])

Invoked with: TensorMap(primary_key="positions") with 3 attributes:
  - labels   : shape={2}, dtype=Int64, device=CPU:0
  - normals  : shape={2, 3}, dtype=Float32, device=CPU:0
  - positions: shape={2, 3}, dtype=Float32, device=CPU:0 (primary)
  (Use . to access attributes, e.g., tensor_map.positions)

## PointCloud attributes setter and getter

In [39]:
pcd = o3d.t.geometry.PointCloud(o3c.Tensor([[0, 0, 0], [1, 1, 1]], o3c.float32))
# Set attributes.
pcd.point.normals = o3c.Tensor([[0, 0, 1], [0, 0, 1]], o3c.float32)
pcd.point.colors = o3c.Tensor([[1, 0, 0], [0, 1, 0]], o3c.float32)
pcd.point.labels = o3c.Tensor([0, 1], o3c.int64)
print(pcd)

# Set by numpy array or python list.
pcd.point.normals = np.array([[0, 0, 1], [0, 0, 1]], dtype=np.float32)
pcd.point.intensity = [0.4, 0.4]
print(pcd)
print('')

# Get attributes.
posisions = pcd.point.positions
print(posisions)
labels = pcd.point.labels
print(labels)

print(pcd.point)

PointCloud on CPU:0 [2 points (Float32)] Attributes: labels (dtype = Int64, shape = {2}), colors (dtype = Float32, shape = {2, 3}), normals (dtype = Float32, shape = {2, 3}).
PointCloud on CPU:0 [2 points (Float32)] Attributes: labels (dtype = Int64, shape = {2}), colors (dtype = Float32, shape = {2, 3}), normals (dtype = Float32, shape = {2, 3}), intensity (dtype = Float64, shape = {2}).

[[0.0 0.0 0.0],
 [1.0 1.0 1.0]]
Tensor[shape={2, 3}, stride={3, 1}, Float32, CPU:0, 0x557367a54960]
[0 1]
Tensor[shape={2}, stride={1}, Int64, CPU:0, 0x557387f77a70]
TensorMap(primary_key="positions") with 5 attributes:
  - colors   : shape={2, 3}, dtype=Float32, device=CPU:0
  - intensity: shape={2}, dtype=Float64, device=CPU:0
  - labels   : shape={2}, dtype=Int64, device=CPU:0
  - normals  : shape={2, 3}, dtype=Float32, device=CPU:0
  - positions: shape={2, 3}, dtype=Float32, device=CPU:0 (primary)
  (Use . to access attributes, e.g., tensor_map.positions)
